In [5]:
!pip install pillow
!pip install ultralytics
!pip install ttkbootstrap

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# Ensure you have the necessary libraries installed
"""
pip install pillow
pip install ultralytics
pip install ttkbootstrap
"""

import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import ImageTk, Image
from ultralytics import YOLO
import ttkbootstrap as ttk
from ttkbootstrap.tooltip import ToolTip

class ObjectDetectionApp:
    def __init__(self, master):
        self.master = master
        master.title("Object Detection App")
        master.geometry("850x850")
        self.style = ttk.Style("cosmo")  # Modern theme

        # Layout setup
        self.main_frame = ttk.Frame(master, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        master.grid_rowconfigure(0, weight=1)
        master.grid_columnconfigure(0, weight=1)
        
        self.left_frame = ttk.Frame(self.main_frame, padding="10")
        self.left_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S), padx=5, pady=5)

        self.right_frame = ttk.Frame(self.main_frame, padding="10")
        self.right_frame.grid(row=0, column=1, sticky=(tk.W, tk.E, tk.N, tk.S), padx=5, pady=5)

        self.main_frame.grid_rowconfigure(0, weight=1)
        self.main_frame.grid_columnconfigure(0, weight=1)
        self.main_frame.grid_columnconfigure(1, weight=1)

        self.image_label = ttk.Label(self.left_frame)
        self.image_label.grid(row=0, column=0, padx=5, pady=5, sticky=(tk.W, tk.E, tk.N, tk.S))

        self.left_frame.grid_rowconfigure(0, weight=1)
        self.left_frame.grid_columnconfigure(0, weight=1)

        # Buttons
        self.detect_button = ttk.Button(self.right_frame, text="Detect Objects", command=self.detect)
        self.detect_button.grid(row=0, column=0, padx=5, pady=5, sticky=(tk.W, tk.E))
        ToolTip(self.detect_button, text="Click to detect objects in the loaded image")

        self.load_button = ttk.Button(self.right_frame, text="Load Image", command=self.load_image_dialog)
        self.load_button.grid(row=1, column=0, padx=5, pady=5, sticky=(tk.W, tk.E))
        ToolTip(self.load_button, text="Click to load an image for detection")

        # Listbox for displaying detected objects summary
        self.object_list_box = tk.Listbox(self.right_frame, height=30, width=40)
        self.object_list_box.grid(row=2, column=0, padx=5, pady=5, sticky=(tk.W, tk.E, tk.N, tk.S))

        self.right_frame.grid_rowconfigure(2, weight=1)
        self.right_frame.grid_columnconfigure(0, weight=1)

        # Status bar
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        self.status_bar = ttk.Label(master, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.grid(row=1, column=0, sticky=(tk.W, tk.E))

        master.grid_rowconfigure(1, weight=0)

        # Load YOLO model
        self.model = YOLO("yolov8x.pt")

    def load_image_dialog(self):
        file_path = filedialog.askopenfilename(initialdir="/", title="Select Image", filetypes=(("Image files", "*.jpg;*.jpeg;*.png"), ("All files", "*.*")))
        if file_path:
            self.load_image(file_path)
            self.status_var.set("Image loaded")

    def load_image(self, file_path):
        self.image_path = file_path
        image = Image.open(file_path)
        image.thumbnail((800, 800), Image.LANCZOS)  # Resize while maintaining aspect ratio
        self.photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=self.photo)
        self.image_label.image = self.photo

    def detect(self):
        if hasattr(self, 'image_path'):
            self.status_var.set("Detecting objects...")
            self.master.update_idletasks()
            results = self.model.predict(self.image_path)
            result1 = results[0]
            detected_image = Image.fromarray(result1.plot()[:, :, ::-1])
            self.display_detected_image(detected_image)
            self.process_detected_objects(results)
            self.status_var.set("Detection complete")
        else:
            messagebox.showinfo("Error", "Please load an image first.")

    def display_detected_image(self, detected_image):
        detected_image.thumbnail((800, 800), Image.LANCZOS)  # Resize while maintaining aspect ratio
        photo = ImageTk.PhotoImage(detected_image)
        self.image_label.config(image=photo)
        self.image_label.image = photo

    def process_detected_objects(self, results):
        result1 = results[0]
        objects_count = {}
        for box in result1.boxes:
            class_id = result1.names[box.cls[0].item()]
            objects_count[class_id] = objects_count.get(class_id, 0) + 1

        self.object_list_box.delete(0, tk.END)
        for obj, count in objects_count.items():
            self.object_list_box.insert(tk.END, f"{count} {obj}{'s' if count > 1 else ''}")

def main():
    root = ttk.Window(themename="cosmo")
    app = ObjectDetectionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
